#### Overview
* plotly: 파이선에서 애니메이션을 사용할 수 있도록 해줌
  * https://plotly.com/python/animations/
* folium기반 시계열 애니메이션:
  * https://justinmorganwilliams.medium.com/how-to-make-a-time-lapse-heat-map-with-folium-using-nyc-bike-share-data-1ccd2e32c2e3

#### Import Libraries

In [25]:
import os
import pandas as pd
import plotly.express as px

#### Data Preparation

##### Fetching

In [3]:
CCTV_CSV_PATH = os.path.join(
    os.getcwd(), '89_data', 'csv', 'road_cctv_info.csv'
)
cctv_df = pd.read_csv(
    CCTV_CSV_PATH, encoding='EUC-KR', header=0, index_col=None,
    names=[
        'admin_nm', 'addr_rd', 'addr_jb', 'purpose', 'cctv_cnt', 
        'pixel', 'direction', 'period', 'date_install', 'tel', 
        'lat', 'lon', 'date_make', 'offer_admin_cd', 'offer_admin_nm'
    ],
    dtype={
        'cctv_cnt':object, 'pixel':object, 'period':object, 
        'lat':float, 'lon':float, 'offer_admin_cd':object
    }, 
    thousands=',', # 천단위 쉼표 제거(?)
)

cctv_df.shape

(50000, 15)

##### Preprocessing

In [8]:
# 서울특별시 데이터만 추출
seoul_cctv_df = cctv_df[cctv_df.admin_nm.str.contains('서울특별시')]
seoul_cctv_df.shape

(7150, 15)

In [9]:
# 위경도, 설치일(date_install) 데이터 중 값이 null인 데이터를 삭제하기 위해 확인
seoul_cctv_df.isna().sum()

admin_nm             0
addr_rd            438
addr_jb            676
purpose              0
cctv_cnt             0
pixel             1177
direction         2971
period             449
date_install       838
tel                  0
lat                 92
lon                107
date_make            0
offer_admin_cd       0
offer_admin_nm       0
dtype: int64

In [11]:
# 위경도, 설치일(date_install) 데이터 중 값이 null인 데이터 삭제

# seoul_cctv_df.dropna(subset=['lat', 'lon'], axis=0, inplace=True)
seoul_cctv_df = seoul_cctv_df.dropna(subset=['lat', 'lon', 'date_install'], axis=0)
seoul_cctv_df.shape

(6224, 15)

In [12]:
# cctv 갯수 데이터 타입 변경
seoul_cctv_df.cctv_cnt = seoul_cctv_df.cctv_cnt.astype(int)

In [13]:
seoul_cctv_df.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm
86,서울특별시 영등포구,"선유로13길 25,서울영문초등학교 사거리앞",선유로13길 25,생활방범,3,NaN,360도 전방면,30,2018-08,02-2670-4067,37.519353,126.886960,2020-05-06,3180000,서울특별시 영등포구
87,서울특별시 영등포구,"문래로4길 4,현대2차APT 202동 옆 서부간선도로 사잇길 육교앞",문래로4길 4,생활방범,2,NaN,360도 전방면,30,2018-08,02-2670-4067,37.520178,126.882110,2020-05-06,3180000,서울특별시 영등포구
88,서울특별시 영등포구,"선유로9나길 22,사거리앞",문래동5가 12 사거리앞,생활방범,3,NaN,360도 전방면,30,2018-08,02-2670-4067,37.515876,126.886114,2020-05-06,3180000,서울특별시 영등포구
89,서울특별시 영등포구,경인로94길 9-13,문래동1가 2-6,생활방범,3,NaN,360도 전방면,30,2018-08,02-2670-4067,37.514473,126.903631,2020-05-06,3180000,서울특별시 영등포구
90,서울특별시 영등포구,"선유서로 31,현대6차APT 602동 근처 창대교회 사거리앞",문래동6가 55-1 현대6차APT 602동 근처 창대감리교회 사거리앞,생활방범,1,NaN,360도 전방면,30,2018-08,02-2670-4067,37.518936,126.883318,2020-05-06,3180000,서울특별시 영등포구


In [20]:
# 년도만 추출해서 정수로 변환
seoul_cctv_df['year_install'] = seoul_cctv_df.date_install.str[:4].astype(int)

In [21]:
# 년도 순으로 데이터 정렬
seoul_cctv_df_sort_year = seoul_cctv_df.sort_values(by='year_install', ascending=True)

In [22]:
seoul_cctv_df_sort_year.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,year_install
48052,서울특별시 도봉구청,서울특별시 도봉구 시루봉로 2 숭미초교 삼거리,NaN,어린이보호,1,NaN,360도 전방면,30,1905-07,02-2091-4273,37.650939,127.028760,2018-10-06,3090000,서울특별시 도봉구,1905
33021,서울특별시 도봉구청,서울특별시 도봉구 창4동 38-4 (대창 어린이 공원),NaN,생활방범,1,NaN,360도 전방면,30,1905-07,02-2091-4273,37.644327,127.051988,2018-10-06,3090000,서울특별시 도봉구,1905
41560,서울특별시 도봉구청,서울특별시 도봉구 도봉로109길 121 대광사 모퉁이,NaN,어린이보호,1,NaN,360도 전방면,30,1905-07,02-2091-4273,37.649540,127.029384,2018-10-06,3090000,서울특별시 도봉구,1905
8111,서울특별시 도봉구청,서울특별시 도봉구 창1동 산31-1번지일대 금성파크빌안쪽(공원녹지과 조경창고 주변),NaN,생활방범,1,NaN,360도 전방면,30,1905-07,02-2091-4273,37.649091,127.039530,2018-10-06,3090000,서울특별시 도봉구,1905
4918,서울특별시 구로구청,서울특별시 구로구 경인로 518,서울특별시 구로구 구로동 636-1,시설물관리,1,41,NaN,30,2003-09,02-860-3437,37.500239,126.876910,2020-10-23,3160000,서울특별시 구로구,2003


In [23]:
# 년도를 인덱스로 지정(인덱스 중복됨)
seoul_cctv_df_sort_year.index = seoul_cctv_df_sort_year.year_install

In [24]:
seoul_cctv_df_sort_year.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,year_install
year_install,,,,,,,,,,,,,,,,
1905,서울특별시 도봉구청,서울특별시 도봉구 시루봉로 2 숭미초교 삼거리,NaN,어린이보호,1,NaN,360도 전방면,30,1905-07,02-2091-4273,37.650939,127.028760,2018-10-06,3090000,서울특별시 도봉구,1905
1905,서울특별시 도봉구청,서울특별시 도봉구 창4동 38-4 (대창 어린이 공원),NaN,생활방범,1,NaN,360도 전방면,30,1905-07,02-2091-4273,37.644327,127.051988,2018-10-06,3090000,서울특별시 도봉구,1905
1905,서울특별시 도봉구청,서울특별시 도봉구 도봉로109길 121 대광사 모퉁이,NaN,어린이보호,1,NaN,360도 전방면,30,1905-07,02-2091-4273,37.649540,127.029384,2018-10-06,3090000,서울특별시 도봉구,1905
1905,서울특별시 도봉구청,서울특별시 도봉구 창1동 산31-1번지일대 금성파크빌안쪽(공원녹지과 조경창고 주변),NaN,생활방범,1,NaN,360도 전방면,30,1905-07,02-2091-4273,37.649091,127.039530,2018-10-06,3090000,서울특별시 도봉구,1905
2003,서울특별시 구로구청,서울특별시 구로구 경인로 518,서울특별시 구로구 구로동 636-1,시설물관리,1,41,NaN,30,2003-09,02-860-3437,37.500239,126.876910,2020-10-23,3160000,서울특별시 구로구,2003


#### Animation

In [26]:
fig = px.scatter_mapbox(
    seoul_cctv_df_sort_year, lat='lat', lon='lon', size_max=15,
    color='cctv_cnt', size='cctv_cnt', 
    color_continuous_scale=px.colors.cyclical.IceFire,
    animation_frame='year_install',
    zoom=10, center={'lat': 37.57, 'lon': 127.0},
    width=950, height=750
)

In [27]:
fig.update_layout(
    mapbox_style='white-bg',
    mapbox_layers=[{
        'below': 'traces',
        'sourcetype': 'raster',
        'sourceattribution': 'vWorld',
        'source': ['http://xdworld.vworld.kr:8080/2d/gray/service/{z}/{x}/{y}.png']
    }]
)

fig.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
fig.show()